In [50]:
import video_processing as vp
import matplotlib.pyplot as plt
import seed
import numpy

## Static global variabales and file names

In [ ]:
HEIGHT = 256
WIDTH = 256
TRUE_FRAMERATE = 28.815

left_blue_frame_file
right_blue_frame_file
left_green_frame_file
right_green_frame_file

left_mouse_processed_file
right_mouse_processed_file

## Correlation matrix function definition

In [52]:
def generate_matrix_two_mice(
    left_mouse_green_frames, 
    right_mouse_green_frames,
    left_seeds,
    right_seeds, 
    title,
    filename,
    radius=5
):
    all_seeds=numpy.zeros((numpy.size(left_seeds)+numpy.size(right_seeds),numpy.shape(left_mouse_green_frames)[0]))
    the_labels = []
    the_positions = []
    cnt=0
    # FIX ME REVERSED ROW AND COL
    left_first_frame[left_seeds[0].bregma.row, left_seeds[0].bregma.col] = 255
    for seed in left_seeds:
        the_labels.append(seed.name+"-L")
        top = seed.row-radius
        bottom = seed.row+radius
        left = seed.col-radius
        right = seed.col+radius
        
        if top < 0:
            top = 0
        if bottom > 256:
            bottom = 256
        if left < 0:
            left = 0
        if right > 256:
            right = 256
        seed.signal = numpy.mean(numpy.mean(left_mouse_green_frames[:, top:bottom, left:right], axis=1), axis=1)
        all_seeds[cnt,:]=seed.signal
        cnt+=1
        left_first_frame[top:bottom, left:right] = 255
        the_positions.append((seed.row, seed.col))
        
    #left_mouse_green_frames[0, :, :].tofile("left.raw")
    # FIX ME REVERESED ROW AND COL
    right_first_frame[right_seeds[0].bregma.row, right_seeds[0].bregma.col] = 255      
    for seed in right_seeds:
        
        the_labels.append(seed.name+"-R")
        top = seed.row-radius
        bottom = seed.row+radius
        left = seed.col-radius
        right = seed.col+radius
        
        if top < 0:
            top = 0
        if bottom > 256:
            bottom = 256
        if left < 0:
            left = 0
        if right > 256:
            right = 256
        seed.signal = numpy.mean(numpy.mean(right_mouse_green_frames[:, top:bottom, left:right], axis=1), axis=1)

        right_first_frame[top:bottom, left:right] = 255
        all_seeds[cnt,:]=seed.signal
        cnt+=1
        
        the_positions.append((seed.row, seed.col))

    
    
    
    #right_mouse_green_frames[0, :, :].tofile("right.raw")
    
    
    # Calculate correlation coefficients
    corr_mat=numpy.corrcoef(all_seeds)

    fig = plt.figure(figsize=(10,11))
    ax = fig.add_subplot(111)
    cax = ax.matshow(corr_mat, interpolation='nearest', vmin=0, vmax=1, cmap='viridis')
    fig.colorbar(cax, fraction=0.046, pad=0.04)
    ax.set_title(title+"\n", y=1.15)
    ax.set_xticks([i for i in range(numpy.size(left_seeds)+numpy.size(right_seeds))])
    ax.set_yticks([i for i in range(numpy.size(left_seeds)+numpy.size(right_seeds))])
    ax.set_xticklabels(the_labels, rotation='vertical')
    ax.set_yticklabels(the_labels)
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111)
    ax2.imshow(left_first_frame, cmap='gray')
    
    fig3 = plt.figure()
    ax3 = fig3.add_subplot(111)
    ax3.imshow(right_first_frame, cmap='gray')
    fig.savefig(filename + ".svg")
    numpy.save(filename +".npy", corr_mat)
    
    plt.show()

## Load single blue and green frames

In [ ]:
l_mouse_blue = numpy.load(left_blue_frame_file)    
r_mouse_blue = numpy.load(right_blue_frame_file)   

l_mouse_green = numpy.load(left_green_frame_file)  
r_mouse_green = numpy.load(right_green_frame_file) 

## Load the array of frames

In [ ]:
l_mouse_frames = vp.extract_RAW_frames(
    filename=left_mouse_processed_file,
    width=WIDTH, 
    height=HEIGHT,
    num_channels=1
)
# mask negative infinities with -1
left_mouse_frames[np.where(lmouse_frames == -np.inf)] = -1

r_mouse_frames = vp.extract_RAW_frames(
    filename=right_mouse_processed_file,
    width=WIDTH, 
    height=HEIGHT,
    num_channels=1
)
# mask negative infinities with -1
r_mouse_frames[np.where(r_mouse_frames == -np.inf)] = -1

## Bregma Selection

In [ ]:
l_bregma = seed.Bregma(l_mouse_blue)

In [ ]:
r_bregma = seed.Bregma(r_mouse_blue)

### Define parameters

In [ ]:
start_interaction_frame = 2872
end_interaction_frame = 6930

# pixels per mm
l_ppmm = 30
r_ppmm = 30

l_first_frame = np.copy(l_mouse_green)
r_first_frame = np.copy(r_mouse_green)

### Produce adjusted (scaled) seed pixel objects

In [ ]:
seeds = [("M1", 1.03 + 1, 1.365),("FL", .16, 2.47), ("HL", -.7, 1.85), 
         ("aBC", -1.36 + .575, 3.35),("pBC", -1.9, 3.35), ("AC", 0 + 1, 0.6), 
         ("RS", -2.8858 + 1, 0.62043),("V1", -4.2678 + .8, 2.5168), ("mPTA", -2.4962, 2.2932), 
         ("lPTA", -2.4962 - 0.3, 3.35 - 0.2),("Un", -1.5, 2.6)]

l_scaled_seeds = seed.generate_scaled_seeds(
    seeds=seeds,
    bregma=l_bregma,
    ppmm=l_ppmm,
    direction ='u'
)

r_scaled_seeds = seed.generate_scaled_seeds(
    seeds=seeds,
    bregma=r_bregma,
    ppmm=r_ppmm,
    direction ='u'
)

## Generate Correlation Matrix

In [ ]:
generate_matrix_two_mice(
    l_mouse_frames[start_interaction_frame:end_interaction_frame],
    r_mouse_frames[start_interaction_frame:end_interaction_frame],
    l_scaled_seeds,
    r_scaled_seeds,
    "title",
    "correlation_matrix"
    radius=5
)